In [1]:
%cd /homes/kb658/yinyang/

/homes/kb658/yinyang


In [2]:
import yaml
import jsonlines

config_path = "/homes/kb658/yinyang/configs/configs_os.yaml"

# Load config file
with open(config_path, 'r') as f:
    configs = yaml.safe_load(f)
    
artifact_folder = configs["raw_data"]["artifact_folder"]
mono_folder = configs["raw_data"]["mono_folder"]
json_folder = configs["raw_data"]["json_folder"]
raw_data_folders = configs["raw_data"]["raw_data_folders"]

In [3]:
import glob

data_path = raw_data_folders['classical']['folder_path']

# Get all the midi files in the data path recursively
midi_files = glob.glob(data_path + '/**/*.midi', recursive=True)

In [4]:
len(midi_files)

1276

In [15]:
from aria.data.midi import MidiDict
from aria.tokenizer import AbsTokenizer

tokenizer = AbsTokenizer()
midi_file_path = "/import/c4dm-datasets/maestro-v3.0.0/2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_07_R1_2018_wav--4.midi" # midi_files[1]
print(midi_file_path)
mid = MidiDict.from_midi(midi_file_path)
tokenized_sequence = tokenizer.tokenize(mid)

/import/c4dm-datasets/maestro-v3.0.0/2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_07_R1_2018_wav--4.midi


In [8]:
# midi_file_path

# # Copy this file to the current directory
# import shutil
# import os

# shutil.copy(midi_file_path, 'test.midi')

In [16]:
len(tokenized_sequence)

48136

In [17]:
tokenized_sequence[0:150]

[('prefix', 'instrument', 'piano'),
 '<S>',
 ('piano', 67, 90),
 ('onset', 0),
 ('dur', 570),
 ('piano', 65, 90),
 ('onset', 560),
 ('dur', 530),
 ('piano', 70, 90),
 ('onset', 1070),
 ('dur', 550),
 ('piano', 72, 90),
 ('onset', 1580),
 ('dur', 250),
 ('piano', 77, 90),
 ('onset', 1820),
 ('dur', 290),
 ('piano', 74, 90),
 ('onset', 2070),
 ('dur', 550),
 ('piano', 72, 90),
 ('onset', 2580),
 ('dur', 280),
 ('piano', 77, 90),
 ('onset', 2830),
 ('dur', 300),
 ('piano', 74, 90),
 ('onset', 3100),
 ('dur', 550),
 ('piano', 70, 90),
 ('onset', 3600),
 ('dur', 550),
 ('piano', 72, 90),
 ('onset', 4120),
 ('dur', 590),
 ('piano', 67, 90),
 ('onset', 4660),
 ('dur', 550),
 '<T>',
 ('piano', 65, 90),
 ('onset', 180),
 ('dur', 580),
 ('piano', 67, 90),
 ('onset', 730),
 ('dur', 500),
 ('piano', 55, 60),
 ('onset', 770),
 ('dur', 470),
 ('piano', 58, 60),
 ('onset', 770),
 ('dur', 470),
 ('piano', 43, 60),
 ('onset', 770),
 ('dur', 470),
 ('piano', 62, 45),
 ('onset', 770),
 ('dur', 460),
 ('p

In [54]:
# Define a function to flatten the tokenized sequence
def flatten(sequence):
    flattened_sequence = []
    note_info = []
    for i in range(len(sequence)):
        if sequence[i] == "<T>":
            flattened_sequence.append(sequence[i])
        if sequence[i][0] == "piano":
            note_info.append(sequence[i][1])
            note_info.append(sequence[i][2])
        elif sequence[i][0] == "onset":
            note_info.append(sequence[i][1])
        elif sequence[i][0] == "dur":
            note_info.append(sequence[i][1])
            flattened_sequence.append(note_info) 
            note_info = []

    return flattened_sequence

def skyline(sequence: list, diff_threshold=50):
    melody = []
    pitch = 0
    velocity = sequence[0][1]
    onset = sequence[0][2]
    duration = sequence[0][3]
    i = 0

    for i in range(len(sequence)):
        if type(sequence[i]) != str:
            current_pitch = sequence[i][0]
            current_velocity = sequence[i][1]
            current_onset = sequence[i][2]
            current_duration = sequence[i][3]

            # Get next note onset
            if i+2 < len(sequence):
                if type(sequence[i+1]) == str and type(sequence[i+2]) == str:
                    next_onset = 5000
                elif type(sequence[i+1]) == str and type(sequence[i+2]) != str:
                    next_onset = sequence[i+2][2]
                else:
                    next_onset = sequence[i+1][2]
            else:
                next_onset = 5000
            
            # Get difference between current and next onset
            diff_next_curr_onset = abs(next_onset - current_onset)

            # Get difference between current and previous onset
            diff_curr_prev_onset = abs(current_onset - onset)

            # Check if current_onset is different from next onset
            if diff_next_curr_onset > diff_threshold:
                # Check if current_onset is different from previous onset and if current_pitch is greater than pitch
                if diff_curr_prev_onset < diff_threshold and current_pitch > pitch:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                elif diff_curr_prev_onset >= diff_threshold:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                else:
                    continue
                # Append the last note
                melody.append(("piano", pitch, velocity))
                melody.append(("onset", onset))
                melody.append(("dur", duration))

            else:
                if current_pitch > pitch:
                    pitch = current_pitch
                    onset = current_onset
                    duration = current_duration
                    velocity = current_velocity
                else:
                    continue
        else:
            melody.append(sequence[i])
    return melody
                
# Call the flatten function
flattened_sequence = flatten(tokenized_sequence)
print(flattened_sequence[0:100])

# Call the skyline function
melody = skyline(flattened_sequence, diff_threshold=30)
melody = tokenized_sequence[0:2] + melody + tokenized_sequence[-2:]
print(melody[0:100], '\n')

[[67, 90, 0, 570], [65, 90, 560, 530], [70, 90, 1070, 550], [72, 90, 1580, 250], [77, 90, 1820, 290], [74, 90, 2070, 550], [72, 90, 2580, 280], [77, 90, 2830, 300], [74, 90, 3100, 550], [70, 90, 3600, 550], [72, 90, 4120, 590], [67, 90, 4660, 550], '<T>', [65, 90, 180, 580], [67, 90, 730, 500], [55, 60, 770, 470], [58, 60, 770, 470], [43, 60, 770, 470], [62, 45, 770, 460], [65, 75, 1240, 530], [45, 60, 1260, 500], [60, 60, 1260, 500], [57, 60, 1270, 490], [53, 60, 1270, 490], [70, 90, 1760, 460], [62, 45, 1770, 70], [58, 75, 1770, 450], [43, 75, 1770, 450], [55, 75, 1780, 100], [72, 90, 2250, 490], [69, 75, 2250, 490], [60, 75, 2270, 470], [41, 75, 2270, 470], [53, 75, 2270, 50], [77, 90, 2500, 240], [74, 90, 2760, 460], [38, 75, 2780, 440], [65, 75, 2780, 440], [50, 75, 2780, 430], [69, 60, 2790, 430], [72, 90, 3240, 490], [60, 75, 3250, 480], [69, 75, 3250, 480], [41, 75, 3250, 470], [53, 75, 3260, 460], [77, 90, 3460, 260], [74, 90, 3740, 460], [58, 75, 3750, 450], [65, 75, 3750, 44

In [19]:
print(tokenized_sequence[1:150])

['<S>', ('piano', 67, 90), ('onset', 0), ('dur', 570), ('piano', 65, 90), ('onset', 560), ('dur', 530), ('piano', 70, 90), ('onset', 1070), ('dur', 550), ('piano', 72, 90), ('onset', 1580), ('dur', 250), ('piano', 77, 90), ('onset', 1820), ('dur', 290), ('piano', 74, 90), ('onset', 2070), ('dur', 550), ('piano', 72, 90), ('onset', 2580), ('dur', 280), ('piano', 77, 90), ('onset', 2830), ('dur', 300), ('piano', 74, 90), ('onset', 3100), ('dur', 550), ('piano', 70, 90), ('onset', 3600), ('dur', 550), ('piano', 72, 90), ('onset', 4120), ('dur', 590), ('piano', 67, 90), ('onset', 4660), ('dur', 550), '<T>', ('piano', 65, 90), ('onset', 180), ('dur', 580), ('piano', 67, 90), ('onset', 730), ('dur', 500), ('piano', 55, 60), ('onset', 770), ('dur', 470), ('piano', 58, 60), ('onset', 770), ('dur', 470), ('piano', 43, 60), ('onset', 770), ('dur', 470), ('piano', 62, 45), ('onset', 770), ('dur', 460), ('piano', 65, 75), ('onset', 1240), ('dur', 530), ('piano', 45, 60), ('onset', 1260), ('dur', 5

In [52]:
print(len(tokenized_sequence))
print(len(melody))

48136
14965


In [55]:
mid_dict = tokenizer.detokenize(melody) # mid_dict is a MidiDict object
mid = mid_dict.to_midi() # mid is a mido.MidiFile
mid.save('test_file.mid')

In [27]:
mid_dict